# Function reference


*NB: This documentation has been carried over from the pre-Jupyter databaker, and expanded on here using runnable code examples. Some of this documentation may not be current.*

In [91]:
from databaker.framework import *

In [92]:
# put your input-output files here
inputfile = "example1.xls"
outputfile = "example1.csv"
previewfile = "preview.html"

tabs = loadxlstabs(inputfile)

Loading example1.xls which has size 7168 bytes
Table names: ['beatles', 'stones']


In [93]:
beatles = tabs[0]

## Constants

### Directions

ABOVE (synonym for UP), BELOW (synonym for DOWN), LEFT, RIGHT

### Special dimensions

* OBS is an observation, a particular piece of data.
* DATAMARKER refer to footnote-like things, for example `(a)` to refer
  to approximate data.
* GEOG is the GSS code, e.g. `E04001323`.
* TIME specifies the period, e.g. a year, month or quarter for which the
  observation applies, e.g. `Feb-Apr 1971`.
* TIMEUNIT is the length of the time period, e.g. `Quarter`.
* Also STATPOP, UNITOFMEASURE, UNITMULTIPLIER, MEASURETYPE, STATUNIT.

### Selection specifiers

How we get from observations to headers:

* CLOSEST (gets the *first* cell in the same column or row as the
  observation in a specified direction);
* DIRECTLY (gets the *closest* cell in the same column or row as the
  observation in a specified direction).

## Bags

### Set operators

Bags are 'sets' of cells - and you can treat them just like Python sets.

Specifically, you can use:

```python
newbag = bigbag | bag_of_cells_we_want  # also .union
newbag = bigbag - bag_of_cells_we_dont  # also .difference
newbag = bag & other_bag                # also .intersection
```


![Bags as sets example](set.jpeg)

In [94]:
# Difference between bags — bag_a - bag_b — gives the cells in bag_a, but not in bag_b.

# NB: This isn't the simplest way of selecting these cells; you could select
# the headers then fill(RIGHT) instead

row_headers = beatles.filter("John").assert_one().expand(DOWN)
# savepreviewhtml(row_headers)
all_value_rows = beatles.filter("John").assert_one().expand(RIGHT).expand(DOWN)
# savepreviewhtml(all_row_cells)
# Finding the difference of all row cells and headers gives just the values. 
savepreviewhtml(all_value_rows - row_headers)

In [95]:
# Union of cells — bag_a | bag_b — gives cells that are in bag_a and bag_b (including those in both).
column_headers = beatles.filter("Cars").assert_one().expand(RIGHT)
all_headers = row_headers | column_headers
savepreviewhtml(all_headers)

In [96]:
# Intersection of cells — bag_a & bag_b — gives cells in both bag_a and bag_b.
cells_in_both_headers = row_headers & column_headers
# This set is empty as no cells are in both sets of headers.
print(cells_in_both_headers)

set()


### bag.filter(_string_)

Return only the cells in the bag with a value of _string_. Typically there's only one, so this is often followed by `.assert_one()`. See later for other options to pass to select.

In [97]:
singer = beatles.filter("Mick")
# No matches, as expected.
print(singer)

set()


In [98]:
# If we want to fail loudly, we can use .assert_one()
# This is useful for guaranteeing a value must be present
# before continuing with the rest of the recipe.
singer = beatles.filter("Mick").assert_one()

NoCellsAssertionError: assert_one() : 0 cells in bag, not 1

In [99]:
# This works without any error, as the expected value is present.
singer = beatles.filter("Paul").assert_one()
print(singer)

{<A5 'Paul'>}


### bag.one\_of(_list-of-selects_) [db]

Return cells in the bag for which any of the selects match. selects are usually strings but could also be _cell-functions_ (see the backend of bag.select, below).

In [100]:
multiple_matches = beatles.one_of(["Paul", "John"])
savepreviewhtml(multiple_matches)

### bag.regex(_regular-expression_) [db]

Return cells whose values match the regular expression.

In [101]:
# Here, we're finding cells that match the regular expression
# or "regex": .*s$
# For this match to succeed, we can have any character, any number of times,
# provided there is a s at the end of it (denoted by $)

# https://regexone.com/ is a nice, easy to follow tutorial starting from scratch.
# https://regex101.com/ is useful for testing out regular expressions
regex_matches = beatles.regex(".*s$")
savepreviewhtml(regex_matches)

### bag.assert_one()

Return the bag unchanged, so long as it contains exactly one cell.
Otherwise crash.

### bag.shift(_x_, _y_)
### bag.shift(_direction_)

Return only cells which are exactly _x_ cells to the right, and _y_ cells down from a cell in the current bag. Either _x_ or _y_ can be negative to look left and up respectively.

There's no guarantee that you'll have the same number of cells afterwards; there might not be a cell at the new position.

You can also use any direction: `UP`, `DOWN`, `LEFT`, `RIGHT`, `ABOVE`, `BELOW`; or pass an `(`_x_`, `_y_`)` tuple.

![Shift example](shift.jpeg)

In [102]:
# Get cells that are shifted two to the right from
# the row headers (which were previously selected above in
# the set operations section).
shifted_right = row_headers.shift(2, 0)
savepreviewhtml(shifted)

### bag.extrude(_x_, _y_)

Get this cell, and all cells within _x_ squares right of it, and _y_ squares below it. _x_ and _y_ can be negative for left and up respectively.

![Extrude example](extrude.jpeg)

### bag.fill(_direction_)
### bag.expand(_direction_)

From each of the cells in the bag, get all cells that are in the specified _direction_. `fill` doesn't include the original cells; `expand` does.
Note that this might get a lot of blank cells: consider using `is_not_blank()`.

![Fill and expand example](fillexpand.jpeg)

It also has a `stop_before` parameter: this takes a function which is tested against each bag; the fill/expand will stop if function(bag) is True. Note that this only works when cells are in a single row or column (specifically; the cells are scanned in book-reading order, left to right and top to bottom; we stop the first time the value is True).

In [103]:
john_and_paul = beatles.filter("Ringo").assert_one() | beatles.filter("John").assert_one()
savepreviewhtml(john_and_paul.fill(RIGHT))

In [104]:
savepreviewhtml(john_and_paul.expand(RIGHT))

### bag.is_number() [DB]

Return only the cells in the bag which have a numeric value.

In [105]:
number_cells = beatles.is_number()
savepreviewhtml(number_cells)

### bag.is_date() [DB]

Return only the cells in the bag which are in one of the following string formats:
```
1999
1999 Q1
Jan 1999
Jan-Mar 1999
```
Note that currently this won't detect cells which are actually recognised by Excel as dates!

In [106]:
date_cell = beatles.is_date()
savepreviewhtml(date_cell)

### bag.spaceprefix(count)

Return cells which have `count` whitespace characters at the start, and then a non-whitespace character.

### bag.is_XXX()
### bag.is\_not\_XXX()

Return only the cells which are / are not a thing.

```python
bag.is_not_bold()
bag.is_strikeout()
```

Options which make sense: `bold`, `italic`, `strikeout`, `underline`, `blank`, `any_border`, `all_border`, `richtext`, `whitespace`.
(is any border lined? are all four borders lined? is this cell richtext?)

In [107]:
bold_cells = beatles.is_bold()
savepreviewhtml(bold_cells)

In [108]:
non_bold_cells = beatles.is_not_bold()
savepreviewhtml(non_bold_cells)

### bag.XXX\_is(_value_)
### bag.XXX\_is\_not(_value_)

Return only the cells for which the property has / doesn't have a particular value.

Options which make sense: `size` (in points), `font_name` (a string).

`bag.font_name_is_not("Comic Sans MS")`

In [109]:
ten_point_cells = beatles.size_is(10)
savepreviewhtml(ten_point_cells)

### bag.value
Returns the value of a single cell; i.e. the bag must contain a single cell only. Otherwise, this will fail with an assert_one() error if multiple cells are in the bag.

In [110]:
print(date_cell.value)

2014.0


### bag.parent()

For any spanned cells, get the top-left cell (which contains all the info).

Non-spanned cells are passed unchanged.

### bag.children()
For any cell that is part of a span, get *all* cells that are parts of the span.

Useful for .fill(DOWN), but note that if the spanned cell is more than one row high,
that'll include some of the spanned cells!

### bag.group(_regular-expression_) [db]

For a single cell, get the matching group from the regular expression.

e.g. if the regular expression was `-(.*)-` and the cell's value was `cat-dog-fish`; this would return `dog`.

In [111]:
train_cell = beatles.filter("Trains")
# A matching group is specified by parentheses: ( )
# For a regular expression that matches the cell's value, using
# a matching group means you only get text captured by the parentheses.
april_weather = train_cell.group("T(.*)s")
print(april_weather)

rain


### bag.waffle(_other-bag_)

Get all cells which have a cell from one bag above them, and the other bag to the side. Note that the two bags are interchangable without changing the output. You can change the direction from its default (DOWN) by specifying `direction=LEFT` or similar.

![Junction extrude example](junctionextrude.jpeg)

### bag.same\_row(_other-bag_)
### bag.same\_col(_other-bag_)

Get cells in this bag which are in the same row/column as a cell in the second.

![Same row example](same_row.jpeg)

In [112]:
john_values = beatles.filter("John").assert_one().fill(RIGHT)
planes_column = beatles.filter("Planes").assert_one().fill(DOWN)
savepreviewhtml(planes_column.same_row(john_row))

### bag.table

The table which these cells are from.

In [113]:
print(singer.table)

{<D3 'Trains'>, <B2 ''>, <A5 'Paul'>, <B1 2014.0>, <C7 5.0>, <A3 ''>, <A2 ''>, <C4 2.0>, <D7 5.0>, <D2 ''>, <A4 'John'>, <B7 2.0>, <C3 'Planes'>, <A6 'Ringo'>, <A7 'George'>, <B4 2.0>, <C6 1.0>, <D1 ''>, <B5 4.0>, <C5 3.0>, <C1 ''>, <A1 'Date'>, <C2 ''>, <D4 1.0>, <B3 'Cars'>, <D6 3.0>, <B6 4.0>, <D5 2.0>}


### bag.by_index(index)

Get cells from this bag by their position in the bag: i.e. `by_index(1)` will get the first.

Can also pass lists of numbers: `by_index([2,4,6,8])` but this might get removed in future,,
in favour of using standard Python indexes `bag[4]`.

**experimental** : might remove and replace with standard pythonic indexes.

In [114]:
print(beatles.by_index(1))
print(beatles.by_index([17,21]))

{<A1 'Date'>}
{<A5 'Paul'>, <A6 'Ringo'>}


## Table only

### table.debug_dimensions() [db]
Adds debug dimensions - giving the table name and Excel cell reference - to the output CSV.

### table.excel\_ref(_ref_) [db]

Get cells from the table via an Excel cell reference: e.g. "G4", "C", "9", "Q5:Z9", "A:C", "9:88".

This is useful as a last resort if there's absolutely no other way of
specifying a particular cell or cells. However, it's preferable to
select cells via dimensions instead as these will be more robust to
spreadsheet change.

For example, if all the cells are shifted right one
in a new version of a spreadsheet, the desired cell will have shifted
too, but the reference in the recipe won't be updated and the recipe
will not behave as intended.

In [115]:
print(beatles.excel_ref("A5"))

{<A5 'Paul'>}


### Shallow magic

These are reasonably detailed internals, but might be useful to know.

### bag.select(_function_)

Get cells from the same table as these cells, based on a function.

The function must take two arguments: a bag cell, and a table cell.

If the function is `True`, the table cell will be in the output.

### bag.select_other(lambda bag_cell, other_cell: foo, other_bag)

As `bag.select` except it inspects another bag rather than the bag's table.

This is useful because you can return cells from only a subset of the table, or use a different table entirely.

### bag.junction(otherbag)

Similar to `bag.waffle`, except it outputs a list of the form

```python
[
    [bag_cell, otherbag_cell, cell_at_junction],
    [bag_cell, otherbag_cell, cell_at_junction],
    ...
]
```

### bag.filter(_cell-function_)

This is the more general form of the 'filter' command.
Return only the cells in the bag for which the _cell-function_ is true. These are typically written
```python
bag.filter(lambda cell: cell.value > 20 and cell.y < 40)
```
or

```python
def large_value_near_top(cell):
    return cell.value > 20 and cell.y < 40
    
bag.filter(large_value_near_top)
```

Optionally, if you install *hamcrest* (`pip install pyhamcrest`) you can use their matching functions on the cell value - see the [hamcrest documentation](http://pyhamcrest.readthedocs.org/en/latest/library/) for the available functions.

```python
bag.filter(hamcrest.equal_to("dog"))
```

Of specific interest is when the _cell-function_ is `re.compile(`_regular-expression_`)`: this is `True` when the regular expression matches the cell's value.  

### bag.glue(_cell-function_, _join-function_, _blank_=True)

**WARNING** : this function modifies cell values and has interesting side-effects. See details below.

Sometimes, what should be one cell in the output is split across multiple cells in the Excel spreadsheet. `.glue` rejoins these cells together.

![Glue example](glue.png)

The bag needs to contain the key cells: those which will contain values once we're done. For each of these cells, the cell_function is called on it, giving a range of ancillary cells which will provide their values to the key cell.

The key cell's value becomes the concatenation of it and the ancillary cells in reading order; the ancillary cells values are then wiped (unless `blank` is set to `False`)

This does not change the formatting properties of the cells: in particular, the cells are not spanned together.

The `join_function` defaults to concatenating the values together with space: a different function can be provided. (i.e. `' '.join`)

It is *strongly* recommended that gluing of cells occurs as early as possible in the `per_tab` function: the results of other functions will change after a call to `.glue`.

Example code for above:

```python
def per_tab(tab):
    # ... code to acquire key_cells, e.g.
    key_cells = tab.filter("All homes")
    key_cells.glue(lambda cell: cell.extrude(0,3))

    # all homes cells have been changed so the 
    # function we used gives us a different answer now
    assert len(tab.filter("All homes")) == 0

    # ... rest of per_tab function as per usual
    tab.filter(re.compile("All homes.*")).dimension('location', DIRECTLY, ABOVE)
    # ...

```

**WARNING** -- Unlike other functions this modifies the values of the cells. Inappropriate use can lead to inconsistent results! -- **WARNING**

Selectors which filter by the values of the affected cells -- like the ones used to select the key cells in the first place -- will return results based on the *modified* values after the use of `.glue`.

It is therefore *strongly* recommended that gluing of cells occurs as early as possible in the `per_tab` function.

Whilst `.glue` can be chained (it returns the bag of key cells provided to it; the bag is unchanged but the values of the cells in it are) it's not recommended since it isn't a 'pure' function.

## Other things that might be useful, maybe.

`scan.py` looks for duplicated sets of dimension values

`bag.pprint`, `.aslist` and `.excel_locations` might be useful ways of displaying the data when debugging.